---
title: "Recitation 1"
bibliography: ../reading_list.bib
---


# Creating Household Income Measures in the CPS-ASEC

In this recitation we're going to work with extracts of the CPS-ASEC from the years (1970,1980,1990,2000,2010,2019). This is a slightly smaller dataset than the version I'm using in class, but will nonetheless capture the same trends.

Our focus in this class will be earned income at the individual and household level, as well as government transfers at the individual and household level. The code below loads the data and creates some variables:

- `EARN` which will be the sum of wages and business income
- `GOV` which will be the sum of welfare, social security income, supplemental security income, the Earned Income Tax Credit (EITC) and the Child Tax Credit (CTC).

Some other things to note:

- These are **limited measures** of income and if you peruse the variables in IPUMS you will find many more categories.
- Wage and business income is subject to [**top coding**](https://cps.ipums.org/cps/topcodes_tables.shtml) and we are not doing anything to solve that issue here, but you should be aware of it. See @Heathcote2023 for one approach to this problem.
- Many of the variables we are analyzing are assigned dummy values (e.g. 9999999) if they are missing or not asked for an individual in the survey ("Not in Universe"). One should normally check to make sure these dummy values don't appear in the final data.
- Our measure of transfers does not include **in-kind** transfers such as Medicaid and Food Stamps (which comprise a very large share of overall transfers).

With that being said, below is code to load the data and create these variables:


In [ ]:
using DataFrames, CSV, StatsPlots, DataFramesMeta, StatsBase
pce = CSV.read("../data/PCE_index.csv",DataFrame)

data = @chain CSV.read("../data/asec_1970_2019.csv",DataFrame) begin
  innerjoin(pce,on=:YEAR)
  @transform begin 
    # fill in some missing values and deflat
    :EITC = 100 * coalesce.(:EITCRED,0.) ./ :PCE
    :CTC = 100 * coalesce.(:ACTCCRD,0.) ./ :PCE
    :SSI = 100 * coalesce.(:INCSSI,0.) ./ :PCE
    :WELF = 100 * :INCWELFR ./ :PCE
    :SS = 100 * :INCSS ./ :PCE
    # create the income measures
    :EARN = 100 * (:INCWAGE .+ :INCBUS) ./ :PCE #<- deflate earnings (2017 base)
    end
    @transform :GOV = :EITC .+ :CTC .+ :SSI .+ :WELF .+ :SS
end

## Constructing a measure of pooled or equivalized income

Each row of thse data corresponds to an individual in a particular year. For some questions, we may instead be interested in per-person (or equivalized) measures of these income variables. The code below constructs those measures by summing over all individuals within a household, where `CPSID` is a unique identifier for each household in the survey.


In [ ]:
# - gov transfers
# - Create a household-level dataset by summing CPSID (the household ID variable)
house = @chain data begin
    @subset .!ismissing.(:CPSID) #<- drop all obs where this variable is missing (all of 1970)
    @select :CPSID :CPSIDP :YEAR :EARN :GOV :SS :SSI :WELF :CTC :EITC
    stack(Not([:CPSID,:CPSIDP,:YEAR]))
    groupby([:YEAR,:CPSID,:variable])
    @combine :value = mean(:value)
    unstack([:YEAR,:CPSID],:variable,:value)
end

Some notes:

- Try commenting out the code that follows the `stack()` command to see how the `stack` and `unstack` functions work here.
- Notice that we are calculating the mean only over individuals 25-64 in the sample (since we dropped everyone else before even pulling the data from IPUMS). Ideally one would want to include other household members as well for this equivalized measure.

## Plotting and decomposing transfers over time

How prominently do government transfers feature in the household budget constraint? And how has this evolved over time? Although our measures of income and transfers are not exhaustive, we can use them to get a partial answer to this question.

The code below calculates average transfers relative to average net income for the poorest 20% of households (as measured by our variable `EARN`) and produces a plot.

In [ ]:
@chain house begin
    groupby(:YEAR)
    @transform :q20 = quantile(:EARN,0.2) #<- keep bottom 20% in each year
    @subset :EARN .< :q20
    groupby(:YEAR)
    @combine begin 
        :gov_share = mean(:GOV)  / (mean(:GOV) .+ mean(:EARN))
    end
    @df _ begin 
        plot(:YEAR,:gov_share,linewidth=3)
        scatter!(:YEAR,:gov_share,legend=false)
    end
end

Note that these fluctuations can be caused by fluctuations in the numerator or the denominator. **An exercise for you** to complete is to verify that these fluctuations are driven by fluctuations in earned income. What event in 2010 could explain the large drop in earned income?


### Sources of government income

Over this time, what sources of government transfers are most prevalent? The code below plots the five sources of transfers that we included in our measure as a fraction of the total in each year. 


In [ ]:
@chain house begin
    groupby(:YEAR)
    @combine  begin 
        :eitc = mean(:EITC)  / mean(:GOV)
        :ssi = mean(:SSI)  / mean(:GOV)
        :welf = mean(:WELF)  / mean(:GOV)
        :ctc = mean(:CTC)  / mean(:GOV)
        :ss = mean(:SS)  / mean(:GOV)
    end
    stack(Not(:YEAR))
    @df _ plot(:YEAR,:value,group = :variable,linewidth = 3.)
end